In [1]:
import numpy as np
import theano
import theano.tensor as T
import scipy.sparse as sp
from theano import sparse
import lasagne
import time
import scipy.stats as stats


/Applications/anaconda/lib/python3.5/site-packages/theano/tensor/signal/downsample.py:6: UserWarning: downsample module has been moved to the theano.tensor.signal.pool module.
  "downsample module has been moved to the theano.tensor.signal.pool module.")


In [2]:
# Generate a matrix from a multivariate normal distribution with low-rank covariance matrix
KTRUE = 10
K = 20
N = 100
D = 50
maxit = 3*KTRUE*(N+D-KTRUE)

np.random.seed(seed=10)

w   = np.random.uniform(low=0.0, high=1.0, size=(D,KTRUE))
var = 0.1
covnp = w.dot(w.T)+var*np.eye(D)

Mnp = np.random.multivariate_normal(np.zeros(D), covnp, N).T

In [3]:
# We use Theano for our model
srng = T.shared_randomstreams.RandomStreams(seed=120)

#Define Theano Variables
Shared = lambda shape,name: theano.shared(value = np.ones(shape,dtype=theano.config.floatX),
                                          name=name,borrow=True) 


In [4]:
# Let Ynp represent our matrix of partial, noisy observations
p      = (maxit/20)/(N*D)
#Masknp = np.random.binomial(N, p, size=(N,D)).T
Masknp = stats.bernoulli.rvs(p, size=(D,N))
Mask   = T.as_tensor_variable(Masknp)
M      = T.as_tensor_variable(Mnp)
Y      = Mask*M
zeroY  = T.as_tensor_variable(np.zeros((D,N)))
zero2  = T.as_tensor_variable(np.zeros((D,D)))
zero   = T.as_tensor_variable(np.zeros(D))
st     = T.sum(T.neq(Y, zeroY), axis = 0)
s      = st.eval()

In [5]:
p

0.042

In [6]:
N*D

5000

In [7]:
#Define variational parameters
mW=Shared((D,K), "mW")
sW=Shared((D,K), "sW")
mr=Shared((K), "mr")
sr=Shared((K), "sr")
mGamma=Shared((1), "mGamma")
sGamma=Shared((1), "sGamma")
mGamma0=Shared((1), "mGamma0")
sGamma0=Shared((1), "sGamma0")
mc0=Shared((1), "mc0")
sc0=Shared((1), "sc0")
msigma=Shared((1), "msigma")
ssigma=Shared((1), "ssigma")

#Define model parameters and other random variables (zY, zK, error, KmatrixRandom)
zW= srng.normal((D,K))
zr= srng.normal([K])
zGamma= srng.normal([1])
zGamma0= srng.normal([1])
zc0= srng.normal([1])
zsigma=srng.normal([1])
zY=srng.normal([D])
zK=srng.normal([K])
KmatrixRandom=srng.uniform(size=(D,D), low=0, high=1000)

#All variables have a log-normal variational posterior
W=T.exp(mW+0.1*sW)
r=T.exp(mr + zr*sr)
Gamma=T.exp(mGamma + zGamma*sGamma)
Gamma0=T.exp(mGamma0 + zGamma0*sGamma0)
c0=T.exp(mc0 + zc0*sc0)
sigma=T.exp(msigma +zsigma*ssigma)


#For data given seqentially we need a different covariance matrix for each yn
WWT=T.dot(W, W.T)
Cov=Shared((D,D), 'Cov')
Cov=WWT+sigma[0]*T.identity_like(WWT)


#Define lists
vParams= [mW, sW, mr, sr, mGamma, sGamma, mGamma0, sGamma0, mc0, sc0, msigma, ssigma]
mParams = [W, r, Gamma, Gamma0, c0, sigma]

#indexlist = Shared([maxit], 'indexlist')
indexlist = theano.shared(value = np.zeros([maxit],dtype=np.int64),
                                          name='indexlist',borrow=True)

Estimates = Shared((maxit, N, D), 'Estimates')

In [8]:
#Define Functions for Variational Inference

def Entropy(vParams):
    
    mW, sW, mr, sr, mGamma, sGamma, mGamma0, sGamma0, mc0, sc0, msigma, ssigma= vParams
    entW      = T.log(T.abs_(sW))+mW
    entr      = T.log(T.abs_(sr))+mr
    entGamma  = T.log(T.abs_(sGamma))+mGamma
    entGamma0 = T.log(T.abs_(sGamma0))+mGamma0
    entc0     = T.log(T.abs_(sc0))+mc0
    entsigma  = T.log(T.abs_(ssigma))+msigma
    entropy   = entW.sum()+entr.sum()+entGamma+entGamma0+entc0+entsigma
    
    return(entropy)



def logJointScanFn2(n, logLikelihood, Y, Cov, s):
    
    idxs          = T.neq(Y[:,n], zero).nonzero()
    y             = Y[:,n][idxs]
    idxs2         = T.neq(T.outer(Y[:,n], Y[:,n]), zero2).nonzero()
    littlecov     = Cov[idxs2].reshape((s[n], s[n]))
    logLikelihood +=(-1/2.0)*T.log(T.nlinalg.Det()(littlecov))-(1/2.0)*T.dot(y.T, T.dot(T.nlinalg.MatrixInverse()(littlecov), y))
    
    return logLikelihood



def LogJ2(mParams,vParams,Y, Cov, s):
    
    mW, sW, mr, sr, mGamma, sGamma, mGamma0, sGamma0, mc0, sc0, msigma, ssigma = vParams
    W, r, Gamma, Gamma0, c0, sigma = mParams
    #LogJt0=time.clock()
    results, updates = theano.scan(fn=logJointScanFn2,
                                   sequences = np.arange(N),
                                   outputs_info=[dict(initial= np.float64(0) ,taps=[-1])],
                                   non_sequences=[Y, Cov, s])
    logJoint  = results[-1]
    logJoint2 = ((D*Gamma*T.log(Gamma))[0]*r).sum()-(D*T.gammaln(Gamma[0]*r)).sum()+((Gamma[0]*r-1)*T.log(W)).sum()-(Gamma[0]*W).sum() + (Gamma0*T.log(c0)-K*T.gammaln(Gamma0/K)+(Gamma0/K-1)[0]*(T.log(r)).sum()-(c0[0]*r).sum()-Gamma-Gamma0-c0)[0]
    logJoint  += logJoint2

    return(logJoint)


    
def ELBO(mParams,vParams, Y, Cov, s):
    
    return(LogJ2(mParams,vParams, Y, Cov, s)+Entropy(vParams)[0])

In [9]:
elbo    = ELBO(mParams,vParams,Y,Cov, s)
entropy = Entropy(vParams)
logJ    = LogJ2(mParams,vParams,Y,Cov, s)

In [10]:
Ytrue = Y.eval()

In [11]:
#Stochastic Gradient Descent

elbo=ELBO(mParams,vParams, Y, Cov, s)
entropy=Entropy(vParams)
logJ=LogJ2(mParams,vParams, Y, Cov, s)
#paramt0=time.clock()
vParamUpdates=lasagne.updates.adadelta(-elbo,vParams)
#paramt1=time.clock()
AdaDeltaStep=theano.function(inputs=[], updates=vParamUpdates)


def SGD(Y, limit,construct_elbo_list, construct_error_list):
    
    #Repeat Code as above to construct the partial covariance matrices, also construct binaryY
    #which is a matrix with a 1 in the (i,j) entry if we have observed that entry
    elbo=ELBO(mParams,vParams, Y, Cov, s)
    entropy=Entropy(vParams)
    logJ=LogJ2(mParams,vParams, Y, Cov, s)
    #paramt0=time.clock()
    vParamUpdates=lasagne.updates.adadelta(-elbo,vParams)
    #paramt1=time.clock()

    
    #%%time
    counter = 0
    ELBOlist = []
    errorlist = []
    keepUpdating = True
    if construct_elbo_list==1:
        while keepUpdating:
            AdaDeltaStep()
            keepUpdating = False if counter>limit else True 
            # if construct_elbo_list==1, estimate ELBO by Monte Carlo every 20 steps
            if counter%20==0:
                ELBOlist.append(np.mean([elbo.eval() for i in range(40)]))
                R = 10
                [y_estimate, sigma_u_o_scan, sigma_ob_inv_scan], updates=theano.scan(fn=MVNormalScan_beta02,
                                              sequences=T.arange(N),
                                              outputs_info=None,
                                              non_sequences=[Y, Mask, Cov, W, zY, zK, s])
                Yest = y_estimate.eval().T
                errorlist.append(np.linalg.norm(Yest- Ytrue))                
            counter +=1
    else:
        while keepUpdating:
            AdaDeltaStep()
            keepUpdating = False if counter>limit else True 
            counter += 1
    print(counter)
    
    return ELBOlist, errorlist

In [12]:
#MVNormalScan constructs our estimate of the entire matrix using conditional multivariate normal

def MVNormalScan_beta02(n, Y, Mask, Cov, W, zY, zK, s):
    
    #construct binaryY_unobs a vector of 1s and 0s where the ith coord is a 1 if we haven't seen the ith coord of y_n    
    binaryY_unobs = T.eq(Y[:,n], zero)
    #construct covariance of the observed entries where the rows/columns with nothing have a 1 on diag (so invertible)
    idxs          = T.neq(Y[:,n], zero).nonzero()
    y             = Y[:,n][idxs]
    idxs2         = T.neq(T.outer(Y[:,n], Y[:,n]), zero2).nonzero()
    littlecov     = Cov[idxs2].reshape((s[n], s[n]))
    littlecov_inv = T.nlinalg.MatrixInverse()(littlecov)
    
    
    #sigma_observed     = T.outer(binaryY[:,n], binaryY[:,n])*Cov+(binaryY_unobs*T.identity_like(Cov))
    sigma_unobs_obs         = (T.outer(binaryY_unobs, T.neq(Y[:,n], zero)))*Cov
    idxs3                   = T.neq(sigma_unobs_obs, zero2).nonzero()
    little_sigma_unobs_obs = sigma_unobs_obs[:,idxs].reshape((D, s[n])) 
    #sigma_observed_inv = T.nlinalg.MatrixInverse()(sigma_observed)
    dummyY             = T.zeros(D)
    
     
    #draw the mean vector dummyY from N(0, WWT+sigma^2I) using computationally fast trick
    dummy_results, dummy_updates= theano.scan(lambda prior_result, sigma, zY, W, zK: 
                                              T.sqrt(sigma)[0]*zY+T.dot(W,zK) + prior_result,
                                              sequences=None,
                                              outputs_info= T.zeros(D),
                                              non_sequences=[sigma, zY, W, zK],
                                              n_steps=R)
    
    dummyY       = dummy_results[-1]
    dummyY       /= R
    dummyY_obs   = dummyY[idxs]
    dummyY_unobs = binaryY_unobs*dummyY
    y_est        = dummyY_unobs + T.dot(T.dot(little_sigma_unobs_obs, littlecov_inv), (y-dummyY_obs))
    y_est        = (y_est*binaryY_unobs)-(1e6)*Mask[:,n]
    #y_est        = y_est*binaryY_unobs + Y[:,n]*Mask[:,n]
    
    return [y_est, sigma_unobs_obs, littlecov_inv]



In [13]:
def MainBandit(count, indexlist, Mask, Y):
    
    limit = 500
    R     = 10
    construct_elbo_list = 0
    construct_error_list = 0
    
    elbolist = SGD(Y, limit,construct_elbo_list, construct_error_list)
    
    [y_estimate, sigma_u_o_scan, sigma_ob_inv_scan], updates=theano.scan(fn=MVNormalScan_beta02,
                                              sequences=T.arange(N),
                                              outputs_info=None,
                                              non_sequences=[Y, Mask, Cov, W, zY, zK, s])

    y_estimate     = y_estimate.T
    [value, index] = T.max_and_argmax(y_estimate, axis=None, keepdims=False)   
    mf             = T.flatten(Mask)
    mf             = T.inc_subtensor(mf[index],1)
    Mask           = mf.reshape((D,N))
    
    indexlist      = T.set_subtensor(indexlist[count], index)
    
      
    return indexlist,Mask

In [14]:
def MainBandit2(count, indexlist, Mask,Y):
    
    limit = 500
    R     = 10
    construct_elbo_list = 0
    
    elbolist = SGD(Y, limit,construct_elbo_list, construct_error_list)
    
    [y_estimate, sigma_u_o_scan, sigma_ob_inv_scan], updates=theano.scan(fn=MVNormalScan_beta02,
                                              sequences=T.arange(N),
                                              outputs_info=None,
                                              non_sequences=[Y, Mask, Cov, W, zY, zK, s])

    y_estimate     = y_estimate.T
    [value, index] = T.max_and_argmax(y_estimate, axis=None, keepdims=False)   
    mf             = T.flatten(Mask)
    mf             = T.inc_subtensor(mf[index],1)
    Mask           = mf.reshape((D,N))
    
    #indexlist      = T.set_subtensor(indexlist[count], index)
    indexlist.append(index.eval()) 
    return indexlist,Mask

In [15]:
def ind2sub(array_shape, ind):
    #ind[ind < 0] = -1
    #ind[ind >= array_shape[0]*array_shape[1]] = -1
    rows = np.floor(ind / array_shape[1])
    cols = ind % array_shape[1]
    return (int(rows), int(cols))

In [16]:
count     = 0
ratings   = []
R         = 5

[indices, Mask_evolve], updates = theano.scan(fn=MainBandit,
                                               sequences = T.arange(maxit),
                                               outputs_info = [indexlist,Mask],
                                               non_sequences = Y)
indexlistnp = indices[-1].eval()

502


In [17]:
"""
indexlist2 = []
count = 0

while count< maxit:
    count += 1
    print(count)
    indexlist2,Mask = MainBandit2(count, indexlist2, Mask,Y)
"""

'\nindexlist2 = []\ncount = 0\n\nwhile count< maxit:\n    count += 1\n    print(count)\n    indexlist2,Mask = MainBandit2(count, indexlist2, Mask,Y)\n'

In [18]:
ratings = []
initial_obs = np.multiply([Masknp!=0], Mnp).flatten()
ratings = initial_obs[initial_obs!=0].tolist()
for i in range(np.size(indexlistnp)):
    [r,c] = ind2sub(np.shape(Mnp), int(indexlistnp[i]))
    ratings.append(Mnp[r,c])

In [19]:
reward = np.cumsum(ratings)

In [20]:
#best = Mnp + 0.00001
#best[Masknp] = 1e-6
#best = best.flatten()
#(np.multiply(Mnp, (1e-6)*Masknp)).flatten()
#best = Mnp.flatten()
best.sort()
best[:] =  best[::-1]
best = np.cumsum(best)

random_reward = np.zeros(np.size(Mnp.flatten()))
for i in range(10):
    #random = Mnp + 0.00001
    #random[Masknp] = 1e-6
    #random =  random.flatten()
    random = (np.multiply(Mnp, (1e-6)*Masknp)).flatten()
    random = np.random.permutation(random)
    random_reward += np.cumsum(random)/10

NameError: name 'best' is not defined

In [ ]:
% pylab inline
import numpy as np
import matplotlib.pyplot as plt

plt.plot(best)
plt.plot(reward)
plt.plot(random_reward)

In [ ]:
plt.plot(best[0:np.size(reward)]-reward)
plt.plot(best-random_reward)

In [ ]:
maxit

In [ ]:
N*D